observemos para qué características se presenta información faltante

In [58]:
import pandas as pd
# Cargar el dataset completo
dataset = pd.read_csv('loan.csv')  # Reemplaza con el nombre de tu archivo

C:\Users\samue\AppData\Local\Temp\ipykernel_1016\251971465.py:3: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('loan.csv')  # Reemplaza con el nombre de tu archivo


In [59]:

pd.set_option('display.max_rows', 76)



# Calcular el porcentaje de entradas no nulas para cada atributo
attribute_completeness = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df = attribute_completeness.reset_index()
attribute_completeness_df.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df = attribute_completeness_df.sort_values(by='Completeness (%)', ascending=False)

# Mostrar el resultado
print(attribute_completeness_df)




                      Attribute  Completeness (%)
0                            id        100.000000
39              total_pymnt_inv        100.000000
23                   addr_state        100.000000
24                          dti        100.000000
32                    revol_bal        100.000000
35          initial_list_status        100.000000
36                    out_prncp        100.000000
1                     member_id        100.000000
38                  total_pymnt        100.000000
40              total_rec_prncp        100.000000
20                      purpose        100.000000
41                total_rec_int        100.000000
42           total_rec_late_fee        100.000000
43                   recoveries        100.000000
44      collection_recovery_fee        100.000000
46              last_pymnt_amnt        100.000000
51                  policy_code        100.000000
52             application_type        100.000000
22                     zip_code        100.000000


se eliminan las columnas para las caracteristicas con una presencia menor o igual al 25%:

In [60]:
# Filtrar columnas con más del 25% de datos presentes
columns_to_keep = attribute_completeness[attribute_completeness > 25].index

# Crear un nuevo dataset con las columnas filtradas
filtered_dataset = dataset[columns_to_keep]

# Guardar el dataset filtrado en un nuevo archivo CSV
filtered_dataset.to_csv('loan_filtrado.csv', index=False)


dataset = pd.read_csv('loan_filtrado.csv')
# Mostrar las columnas eliminadas
removed_columns = attribute_completeness[attribute_completeness <= 25].index
print(f"Columnas eliminadas: {list(removed_columns)}")

Columnas eliminadas: ['desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


In [61]:

pd.set_option('display.max_rows', 76)

# Calcular el porcentaje de entradas no nulas para cada atributo
attribute_completeness = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df = attribute_completeness.reset_index()
attribute_completeness_df.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df = attribute_completeness_df.sort_values(by='Completeness (%)', ascending=False)


# Mostrar el resultado
print(attribute_completeness_df)




                     Attribute  Completeness (%)
0                           id        100.000000
40          total_rec_late_fee        100.000000
44             last_pymnt_amnt        100.000000
21                    zip_code        100.000000
22                  addr_state        100.000000
23                         dti        100.000000
42     collection_recovery_fee        100.000000
41                  recoveries        100.000000
1                    member_id        100.000000
18                         url        100.000000
39               total_rec_int        100.000000
38             total_rec_prncp        100.000000
30                   revol_bal        100.000000
37             total_pymnt_inv        100.000000
36                 total_pymnt        100.000000
33         initial_list_status        100.000000
19                     purpose        100.000000
17                  pymnt_plan        100.000000
35               out_prncp_inv        100.000000
8                   

se eliminan caracteristicas que por su descripción o caracteristicas no son útiles para el modelo

#  **Lista de Columnas Eliminadas y Razones de Eliminación**

En este proyecto de predicción de incumplimiento de crédito, hemos eliminado varias características que no aportan valor predictivo o que introducen sesgos en el modelo. A continuación, se explican las razones específicas para cada variable eliminada.

---

## **1️ Características Irrelevantes**
Estas características no aportan información útil para la predicción del incumplimiento de crédito.

- **`desc` y `title`**  
  - Son descripciones textuales proporcionadas por los clientes.  
  - No tienen un formato estructurado, lo que las hace difíciles de procesar.  
  - `title` es redundante con `purpose`, que ya clasifica el propósito del préstamo.  

- **`url`**  
  - Contiene enlaces a detalles del préstamo, sin valor predictivo.  

- **`policy_code`**  
  - Código interno del prestamista, irrelevante para el análisis.  

- **`member_id` y `id`**  
  - Son identificadores únicos de clientes y préstamos, sin relación con el incumplimiento.  

---

## **2️ Características con Alta Cardinalidad**
Las siguientes variables tienen demasiados valores únicos, lo que dificulta su uso sin aumentar demasiado la complejidad del modelo.

- **`emp_title`**  
  - Representa el cargo laboral del solicitante.  
  - Tiene más de **300,000 valores únicos**, lo que la hace difícil de procesar sin técnicas avanzadas de reducción de dimensionalidad.  

- **`zip_code`**  
  - Representa el código postal del cliente.  
  - Tiene una alta cardinalidad y baja correlación con el incumplimiento.  

---

## **3️ Características con Problemas Temporales**
Estas variables dependen del momento en el que se consulta el dataset, lo que podría generar sesgos en el modelo.

- **`next_pymnt_d`, `last_pymnt_d`, `last_credit_pull_d`, `issue_d`, `earliest_cr_line`**  
  - Contienen fechas que pueden cambiar con el tiempo.  
  - Introducen dependencia temporal en el modelo, haciendo que las predicciones sean inconsistentes.  

---

## **4️ Características Disponibles Solo Después del Crédito**
Estas variables contienen información que solo se conoce **después de la aprobación o ejecución del crédito**, lo que introduce fuga de datos.

- **`collection_recovery_fee`, `recoveries`**  
  - Representan montos recuperados después del incumplimiento.  
  - No son útiles porque solo están disponibles **después del evento de impago**.  

- **`last_pymnt_amnt`, `total_rec_prncp`, `total_rec_int`, `total_rec_late_fee`, `out_prncp`, `out_prncp_inv`**  
  - Son montos relacionados con pagos que solo se conocen **después de que se ha realizado el crédito**.  

---

## **5️ Características Prospectivas (No Disponibles en la Solicitud)**
Estas variables dependen de la aprobación del préstamo y no pueden usarse para predecir incumplimiento **antes** de la aprobación.

- **`int_rate`, `installment`, `term`, `purpose`**  
  - Son características definidas **por la entidad financiera tras la aprobación del crédito**.  
  - No se pueden usar en el momento de la solicitud.  

- **`initial_list_status`**  
  - Propiedad definida por la entidad financiera, no por el solicitante.  

---

## **6️ Características Basadas en Historial Crediticio**
Se eliminan estas variables porque asumimos que el solicitante **podría estar accediendo a su primer crédito**, por lo que su historial previo no está disponible.

- **`dti`, `inq_last_6mths`, `pub_rec`, `revol_bal`, `revol_util`, `acc_now_delinq`, `tot_coll_amt`, `collections_12_mths_ex_med`, `total_rev_hi_lim`**  

---

## **7️ Características con Baja Variabilidad**
Variables que tienen valores casi constantes y no aportan información relevante al modelo.

- **`pymnt_plan`**  
  - La gran mayoría de las observaciones tienen el valor `"n"`, lo que la hace poco útil.  

---

## **8️ Otras Características Eliminadas**
- **`sub_grade` y `grade`**  
  - Son calificaciones asignadas por la entidad financiera, **no disponibles antes de la aprobación del crédito**.  

- **`verification_status`**  
  - Propiedad determinada por la entidad financiera, no por el cliente.  

- **`application_type`**  
  - Se busca predecir la probabilidad de incumplimiento de **una persona**, no de un grupo.  



In [64]:
# Lista de columnas a eliminar
columns_to_remove = [
    'addr_state', 
    'collection_recovery_fee',  # solo está disponible después de que el evento de incumplimiento haya ocurrido
    'desc',  # irrelevante
    'title',  # irrelevante
    'zip_code',  # irrelevante
    'policy_code', #irrelevante
    'member_id', # irrelevante
    'id', # irrelevante
    'url', # irrelevante
    'next_pymnt_d', # posibles problemas debido a fechas
    'issue_d',  # posibles problemas debido a fechas
    'emp_title',  # alta cardinalidad
    'earliest_cr_line', # posibles problemas debido a fechas
    'pymnt_plan', # poca variabilidad, solo 10 entradas en para "y", el resto para "n", poca correlacion con objetivo
    'last_pymnt_d', # posibles problemas debido a fechas
    'last_credit_pull_d', # posibles problemas debido a fechas
    'recoveries', # solo está disponible después de que el evento de incumplimiento haya ocurrido
    'sub_grade', # calificación dada por entidades financieras, no posible de conocer
    'grade', # calificación dada por entidades financieras, no posible de conocer
    'verification_status', # propiedad dada por entidad financiera, no depende del posible cliente
    'dti', # se asume que el cliente no tiene deudas diferentes a la que solicita
    'inq_last_6mths', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'pub_rec', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'revol_bal', # imposible de conocer antes del crédito
    'revol_util',  # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'last_pymnt_amnt', # solo está disponible después de que se haya realizado el crédito
    'acc_now_delinq', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'tot_coll_amt', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'total_rec_prncp', # solo está disponible después de que se haya realizado el crédito
    'total_rec_int', # solo está disponible después de que se haya realizado el crédito
    'total_rec_late_fee', # solo está disponible después de que se haya realizado el crédito
    'collections_12_mths_ex_med', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'total_rev_hi_lim', # se asume que no se conoce el historial crediticio del solicitante, ya que podría ser su primer crédito
    'out_prncp', # solo está disponible después de que se haya realizado el crédito
    'initial_list_status', # propiedad dada por entidad financiera, no depende del posible cliente
    'int_rate', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'installment', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'purpose', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'term', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'out_prncp_inv', # solo está disponible después de que se haya realizado el crédito
    'application_type', # se intenta hallar la probabilidad de que una persona incumpla, no un grupo de personas
    'loan_amnt', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'funded_amnt', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'funded_amnt_inv', # caracteristica prospectiva, no se conoce antes de que el crédito sea aprobado
    'total_pymnt_inv', # solo está disponible después de que se haya realizado el crédito
    'total_pymnt', # solo está disponible después de que se haya realizado el crédito
]

# Eliminar las columnas especificadas directamente en el dataset filtrado
dataset.drop(columns=columns_to_remove, errors='ignore', inplace=True)

# Guardar el dataset sobrescrito en el mismo archivo
dataset.to_csv('loan_filtrado.csv', index=False)

# Mostrar las columnas eliminadas
print(f"Columnas eliminadas: {columns_to_remove}")
print("El dataset ha sido sobrescrito en 'loan_filtrado.csv'")


Columnas eliminadas: ['addr_state', 'collection_recovery_fee', 'desc', 'title', 'zip_code', 'policy_code', 'member_id', 'id', 'url', 'next_pymnt_d', 'issue_d', 'emp_title', 'earliest_cr_line', 'pymnt_plan', 'last_pymnt_d', 'last_credit_pull_d', 'recoveries', 'sub_grade', 'grade', 'verification_status', 'dti', 'inq_last_6mths', 'pub_rec', 'revol_bal', 'revol_util', 'last_pymnt_amnt', 'acc_now_delinq', 'tot_coll_amt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'collections_12_mths_ex_med', 'total_rev_hi_lim', 'out_prncp', 'initial_list_status', 'int_rate', 'installment', 'purpose', 'term', 'out_prncp_inv', 'application_type', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'total_pymnt_inv', 'total_pymnt']
El dataset ha sido sobrescrito en 'loan_filtrado.csv'


In [65]:
print(dataset.columns)

Index(['emp_length', 'home_ownership', 'annual_inc', 'loan_status',
       'delinq_2yrs', 'mths_since_last_delinq', 'open_acc', 'total_acc',
       'tot_cur_bal'],
      dtype='object')


Se revisa nuevamente el porcentaje de completitud de cada característica en el dataset para revisar información faltante

In [66]:
attribute_completeness1 = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df1 = attribute_completeness1.reset_index()
attribute_completeness_df1.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df1 = attribute_completeness_df1.sort_values(by='Completeness (%)', ascending=False)
print(attribute_completeness_df1)

                Attribute  Completeness (%)
1          home_ownership        100.000000
3             loan_status        100.000000
2              annual_inc         99.999549
4             delinq_2yrs         99.996732
6                open_acc         99.996732
7               total_acc         99.996732
0              emp_length         94.948607
8             tot_cur_bal         92.080498
5  mths_since_last_delinq         48.802935


se observa que la característica  mths_since_last_delinq es la única con mucha información faltante, así que se decide retirar ya que no es claro si un valor NaN significa 0, se retira para no correr riesgos.

In [67]:
# Lista de columnas a eliminar
columns_to_remove = [
    'mths_since_last_delinq'
]

# Eliminar las columnas directamente en el dataset
dataset.drop(columns=columns_to_remove, errors='ignore', inplace=True)

# Guardar el dataset modificado en el mismo archivo
dataset.to_csv('loan_filtrado.csv', index=False)

print(f"Columnas eliminadas: {columns_to_remove}")
print("El dataset modificado ha sido sobrescrito en 'loan_filtrado.csv'")


Columnas eliminadas: ['mths_since_last_delinq']
El dataset modificado ha sido sobrescrito en 'loan_filtrado.csv'


después de revisar las variables categoricas, se decide modificar emp_length para ser numérica

In [49]:
import pandas as pd

# Cargar el dataset

# Función corregida para convertir la columna emp_length a enteros
def emp_length_converter(df, column):
    df[column] = df[column].str.replace(r'\+ years', '', regex=True)  # Eliminar '+ years'
    df[column] = df[column].str.replace(r'< 1 year', '0', regex=True)  # Reemplazar '< 1 year' por '0'
    df[column] = df[column].str.replace(r' years', '', regex=True)    # Eliminar ' years'
    df[column] = df[column].str.replace(r' year', '', regex=True)     # Eliminar ' year'
    df[column] = df[column].str.replace(r'\+', '', regex=True)        # Eliminar cualquier '+'
    df[column] = pd.to_numeric(df[column], errors='coerce')           # Convertir a numérico (NaN si falla)


# Aplicar la conversión
emp_length_converter(dataset, 'emp_length')


# Guardar el dataset modificado
dataset.to_csv('loan_filtrado.csv', index=False)
print("El dataset modificado ha sido guardado como 'loan_filtrado.csv'.")


El dataset modificado ha sido guardado como 'loan_filtrado.csv'.


### Clasificación de `loan_status` basada en el comportamiento de pago

Para diferenciar entre clientes que no incumplen y que sí incumplen, se redefine la variable `loan_status` de la siguiente manera:

#### **Clientes que incumplen (0)**
- **Fully Paid**: Crédito completamente pagado.
- **Does not meet the credit policy. Status: Fully Paid**: No cumplió con la política, pero fue pagado.

#### **Clientes que no incumplen (1)**
- **Charged Off**: Crédito declarado incobrable.
- **Default**: Crédito en impago definitivo.
- **Does not meet the credit policy. Status: Charged Off**: No cumplió con la política y no fue pagado.

#### **Casos Indeterminados (NaN)**
No permiten determinar si el cliente es bueno o malo y pueden ser excluidos o tratados según el análisis:
- **Current**: Crédito al día, sin certeza de incumplimiento futuro.
- **Late (16-30 days)**: Atraso menor a 30 días, sin indicar incumplimiento definitivo.
- **Late (31-120 days)**: Atraso prolongado. Puede considerarse incumplido o indeterminado.
- **Issued**: Crédito aprobado sin historial de pago.
- **In Grace Period**: Crédito en periodo de gracia, sin evidencia de incumplimiento.


Este criterio alinea la variable objetivo con la identificación real del riesgo de crédito.


In [69]:
import pandas as pd
import numpy as np

# Función para transformar la columna 'loan_status' correctamente
def transform_loan_status(df, column):
    # Diccionario de mapeo
    status_mapping = {
        "Fully Paid": 0,
        "Does not meet the credit policy. Status:Fully Paid": 0,
        "Charged Off": 1,
        "Default": 1,
        "Late (31-120 days)": 1,
        "Does not meet the credit policy. Status:Charged Off": 1,
        "Current": np.nan,  # No se pueden clasificar con certeza
        "Issued": np.nan,  # No hay información de pagos
        "In Grace Period": np.nan,  # Aún no han tenido oportunidad de incumplir
        "Late (16-30 days)": np.nan,  # Aún es incierto si pagarán o no
    }

    # Aplicar la transformación usando el diccionario
    df[column] = df[column].map(status_mapping)

    return df

# Cargar el dataset )
dataset = pd.read_csv('loan_filtrado.csv')

# Aplicar la transformación a 'loan_status'
dataset = transform_loan_status(dataset, 'loan_status')

# Verificar los cambios
print(dataset['loan_status'].value_counts(dropna=False))

# Guardar el dataset modificado sin las filas con valores NA en la variable objetivo
dataset_cleaned = dataset.dropna(subset=['loan_status'])
dataset_cleaned.to_csv('cleaned_dataset.csv', index=False)
print("El dataset modificado ha sido guardado como 'cleanded_dataset.csv'.")


loan_status
NaN    618849
0.0    209711
1.0     58819
Name: count, dtype: int64
El dataset modificado ha sido guardado como 'cleanded_dataset.csv'.


se lleva a cabo un test de chi cuadrado para identificar las variables categóricas más relevantes.
todas tienen valor P muy bajo, lo que significa que todas aportan información útil al modelo.

In [70]:
import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder


dataset = pd.read_csv('cleaned_dataset.csv')  

# Specify the target column (binary classification)
target_column = 'loan_status'  # Replace with the actual name of your target column

# Ensure the target column is binary
if len(dataset[target_column].unique()) != 2:
    raise ValueError("The target column is not binary. Ensure it contains only two unique values.")

# Identify categorical features
categorical_columns = dataset.select_dtypes(include=['object']).columns.tolist()

# Remove the target column from the list of categorical features (if it's included)
if target_column in categorical_columns:
    categorical_columns.remove(target_column)

# Encode categorical features and target variable
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column].astype(str))
    label_encoders[column] = le

# Encode the target column if necessary
if dataset[target_column].dtype == 'object':
    dataset[target_column] = LabelEncoder().fit_transform(dataset[target_column])

# Perform Chi-squared test
chi2_values, p_values = chi2(dataset[categorical_columns], dataset[target_column])

# Create a DataFrame with the results
chi2_results = pd.DataFrame({
    'Feature': categorical_columns,
    'Chi2 Value': chi2_values,
    'P-Value': p_values
}).sort_values(by='Chi2 Value', ascending=False)

# Display the results
print(chi2_results)

# Save the results to a CSV file
chi2_results.to_csv('chi2_feature_selection.csv', index=False)
print("Chi-squared test results saved to 'chi2_feature_selection.csv'.")


          Feature   Chi2 Value        P-Value
1  home_ownership  1081.692497  3.147359e-237
0      emp_length   633.892568  7.115864e-140
Chi-squared test results saved to 'chi2_feature_selection.csv'.


ahora se eliminan las filas que tengan al menos un campo con información faltante

In [71]:
# Eliminar las filas con al menos un NaN

dataset = dataset.dropna()
print(f"Total de filas después de eliminar las filas con NaN: {dataset.shape[0]}")

  #  Guardar el dataset limpio s
dataset.to_csv('cleaned_dataset.csv', index=False)
print("El dataset limpio ha sido guardado como 'cleaned_dataset.csv'.")

Total de filas después de eliminar las filas con NaN: 201934
El dataset limpio ha sido guardado como 'cleaned_dataset.csv'.


**CARACTERISTICAS A UTILIZAR**

In [1]:
print(dataset.dtypes)

NameError: name 'dataset' is not defined